In [27]:
import pandas as pd
import numpy as np

In [28]:
df = pd.read_csv('terraz_leads_marcados.csv')
df.head()

/tmp/ipykernel_10818/471679873.py:1: DtypeWarning: Columns (17,29,30,37,38,45,46,48,50,61) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('terraz_leads_marcados.csv')


,ID,Etapa,Nome do Lead,Primeiro nome,Fonte,Telefone de trabalho,Email de trabalho,Responsável,Criado por,Modificado,...,Brognoli,Terraz,Comercial,Missing.1,Moradia,Autumn,Spring,Summer,Winter,Cluster
0,642984,Agendamento,Lucas Mazagão,Lucas Mazagão,Brognoli_init,NaN,NaN,Brognoli Negócios Imobiliários,Humano,30/12/2022 09:35,...,0,1,0,0,1,0,0,0,1,0
1,642982,Lead,Fernanda Luíza Ferrari,Fernanda Luíza Ferrari,Webpage,5554991509530,ferrari.fernandal@gmail.com,Brognoli Negócios Imobiliários,Humano,31/12/2021 20:27,...,1,0,0,1,0,0,0,0,1,0
2,642980,Desistiu,Fernanda Luíza Ferrari,Fernanda Luíza Ferrari,Webpage,5554991509530,ferrari.fernandal@gmail.com,Brognoli Negócios Imobiliários,Humano,06/01/2022 08:24,...,1,0,0,1,0,0,0,0,1,0
3,642978,Agendamento,Rayssa Scherer Centenaro,Rayssa Scherer Centenaro,Webpage,5547992271499,rah.rc26@gmail.com,Brognoli Negócios Imobiliários,Humano,30/12/2022 09:34,...,0,1,0,0,1,0,0,0,1,0
4,642972,Agendamento,Viviane Pacheco,Viviane Pacheco,Brognoli_init,5548999081844,vivianepacheco0109@gmail.com,Brognoli Negócios Imobiliários,Humano,30/12/2022 09:34,...,1,0,0,0,1,0,0,0,1,0


In [29]:
df['Data e hora da visita'] = pd.to_datetime(df['Data e hora da visita'], format='%d/%m/%Y %H:%M', errors='coerce')
df['dia'] = df['Data e hora da visita'].dt.day
df['mes'] = df['Data e hora da visita'].dt.month
df['ano'] = df['Data e hora da visita'].dt.year
df['dia_da_semana'] = df['Data e hora da visita'].dt.day_of_week

season = (df['Data e hora da visita'].dt.month % 12 + 3) // 3

seasons = {
    1: 'Winter',
    2: 'Spring',
    3: 'Summer',
    4: 'Autumn'
}

df['season'] = season.map(seasons)
df['hora_visita'] = df['Data e hora da visita'].dt.hour
df['minuto_visita'] = df['Data e hora da visita'].dt.minute

In [30]:
num_cols = ['Total', 'dia', 'mes', 'ano', 'dia_da_semana', 'hora_visita', 'minuto_visita', 'visited', 'desistiu',
            'criado_dias', 'Agendamento', 'Atendimento SDR', 'Desistiu', 'Lead', 'Solicitação de Visita', 'Brognoli_init', 'Webpage',
            'Bot.1', 'Agência', 'Blip', 'Chamada', 'Missing', 'Woliver', 'Brognoli', 'Terraz', 'Comercial', 'Missing.1',
            'Moradia', 'Autumn', 'Spring', 'Summer', 'Winter', 'Cluster']
final_cols = ['Email de trabalho', 'CPF do locatário'] + num_cols

In [31]:
df_avg = df[num_cols].groupby('Cluster').mean()
df_avg.columns = [f'avg_{x}' for x in df_avg.columns]

In [32]:
df_sd = df[num_cols].groupby('Cluster').std()
df_sd.columns = [f'sd_{x}' for x in df_sd.columns]

In [33]:
df_sum = df[num_cols].groupby('Cluster').sum()
df_sum.columns = [f'sum_{x}' for x in df_sum.columns]

In [34]:
df_metrics = df_avg.merge(df_sd, on='Cluster').\
    merge(df_sum, on='Cluster')

In [35]:
with pd.ExcelWriter('Leads Marcados.xlsx') as writer:
    df[final_cols].to_excel(writer, sheet_name='Dados Brutos')
    df_metrics.to_excel(writer, sheet_name='Metricas')